# GT98 revisited


In [1]:
using Symbolics

#using ModelingToolkit
# https://stackoverflow.com/questions/70706263/how-to-convert-reduce-output-to-julia-symbolic-expression

Here I need to write Symbolic integration (analytical one) since all the julia packages are not working for me properly!

In [100]:
function integrateTaylorPolynomials(eq::Num, x::Num)
    # this function works only for (positive) polynomials 
    #
    # the function needs only Symbolics.jl
    #
    # Since the Symbolics.coeff does not support expression with division, lowestNegativeOrder::Int option should be implemented afterwards

    lowestNegativeOrder::Int=0
    eq = simplify(eq;expand=true,polynorm=true)
    eqval=Symbolics.value(eq)
    

    # The above statement is to switch off the negative orders integral
   
    if Symbolics.isdiv(eqval)
        eq=eqval.num 
        denominator=eqval.den 
    else 
        denominator =1
    end
    highestOrder=Symbolics.degree(eq,x)
   
    tmpeq = eq
    neweq = 0
    if highestOrder>0
        for i in 1:highestOrder
            old_coef=Symbolics.coeff(eq,x^i)
            tmpeq-=old_coef*x^i
            new_coef=old_coef//(i+1)
            neweq+=new_coef*x^(i+1)        
        end
    end
    
    if lowestNegativeOrder<0
        old_coef=Symbolics.coeff(eq,x^(-1))
        tmpeq-=old_coef*x^(-1)
        new_coef=old_coef
        neweq+=new_coef*log(x)

        if lowestNegativeOrder<1
            for i in lowestNegativeOrder:-2
                old_coef=Symbolics.coeff(eq,x^i)
                tmpeq-=old_coef*x^i
                new_coef=old_coef//(i+1)
                neweq+=new_coef*x^(i+1)
            end
        end
    end

    # the rest of tmpeq should be the constant
    neweq+=tmpeq*x
    neweq/=denominator
    return simplify(neweq;expand=true)
end

integrateTaylorPolynomials (generic function with 1 method)

In [101]:
function ignoreHigherOrderTerms(eq::Num,x::Num,highestOrderForConsideration::Int)
    eq = simplify(eq;expand=true,polynorm=true)
    eqval=Symbolics.value(eq)


    # The above statement is to switch off the negative orders integral
    
    if Symbolics.isdiv(eqval)
        eq=eqval.num 
        denominator=eqval.den 
    else 
        denominator =1
    end
    highestOrder=Symbolics.degree(eq,x)

    highestOrderForConsideration > 0 || error("highestOrderForConsideration should be positive")
    highestOrderForConsideration<highestOrder || return eq/denominator
    tmpeq=eq
    for i in highestOrderForConsideration+1:highestOrder

        old_coef=Symbolics.coeff(eq,x^i)

        tmpeq-=old_coef*x^i
    end
    neweq=tmpeq/denominator
    return simplify(neweq; expand=true)
end
        

ignoreHigherOrderTerms (generic function with 1 method)

In [102]:
@variables x  y z Δx
f=(3*x^3+x^2+y*z*x+z/Δx)^2+x
simplify(f;expand=true)
#Symbolics.degree(f,x)


(z^2 + x*(Δx^2) + (2//1)*(x^2)*z*Δx + (6//1)*(x^3)*z*Δx + (2//1)*x*y*(z^2)*Δx + (x^4)*(Δx^2) + (6//1)*(x^5)*(Δx^2) + (2//1)*(x^3)*y*z*(Δx^2) + (9//1)*(x^6)*(Δx^2) + (6//1)*(x^4)*y*z*(Δx^2) + (x^2)*(y^2)*(z^2)*(Δx^2)) / (Δx^2)

In [103]:
ignoreHigherOrderTerms(1//12*f,Δx,1)

((1//12)*(z^2) + (1//6)*(x^2)*z*Δx + (1//2)*(x^3)*z*Δx + (1//6)*x*y*(z^2)*Δx) / (Δx^2)

In [104]:
integrateTaylorPolynomials(1//12*f,x)

((1//12)*x*(z^2) + (1//24)*(x^2)*(Δx^2) + (1//18)*(x^3)*z*Δx + (1//8)*(x^4)*z*Δx + (1//12)*(x^2)*y*(z^2)*Δx + (1//60)*(x^5)*(Δx^2) + (1//12)*(x^6)*(Δx^2) + (1//24)*(x^4)*y*z*(Δx^2) + (3//28)*(x^7)*(Δx^2) + (1//10)*(x^5)*y*z*(Δx^2) + (1//36)*(x^3)*(y^2)*(z^2)*(Δx^2)) / (Δx^2)

In [105]:
# SymbolicNumericIntegration usage 
#@variables x a b Δx A B
#F, dummy, dummy2=integrate(cosh(a * x) * exp(b * x), x; symbolic = true)
#f(x,xa,xb)
#G, dummy, dummy2=integrate(F,x; symbolic=true)
#substitute(G, Dict([x=>Δx,a=>A,b=>B]))

We are looking for the expression for 
\begin{equation}
\int dt \psi^{(n)} \int dx \phi^{(m)} \left [ \rho (x) \frac{\partial^2}{\partial t^2} u(x,t) - \frac{\partial}{\partial x} \left ( \mu (x) \frac{\partial}{\partial x} u(x,t) \right ) \right ]
\end{equation}
as the inner product:

\begin{equation}
\sum_{m^\prime = m-1}^{m+1} \sum_{n^\prime = n-1}^{n+1} A_{m^\prime n^\prime} u (x_{m^\prime}, t_{n^\prime})
\end{equation}



We use this integration by parts in order to prepare the spatial and temporal integration symbolically.

\begin{equation}
\int_{y_1}^{y_2} dy f(y) w(y) = \left [F(y) w(y) \right ]_{y_1}^{y_2} - \int_{y_1}^{y_2} dy \frac{\partial w(y)}{\partial y} F(y)
\end{equation}
with
\begin{equation}
F(y)=\int dy f(y)
\end{equation}
Since $F(y)$ can be obtained analytically with the aid of Taylor expansion, this symbolic computation is feasible.

If we use spline functions, the above equation can be further developed as:
\begin{align}
\int_{y_1}^{y_2} dy f(y) w(y) &=& \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} - \int_{y_1}^{y_2} dy  F^{(1)}(y) \frac{\partial w(y)}{\partial y} \\
&=& \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} -  \left [F^{(2)}(y) \frac{\partial w(y)}{\partial y} \right ]_{y_1}^{y_2} + \int_{y_1}^{y_2} dy F^{(2)}(y) \frac{\partial^2 w(y)}{\partial y^2} \\
&=& \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} -  \left [F^{(2)}(y) \frac{\partial w(y)}{\partial y} \right ]_{y_1}^{y_2} + \left [F^{(3)}(y) \frac{\partial^2 w(y)}{\partial y^2} \right ]_{y_1}^{y_2} - \cdots 
\end{align}

# specific simplification
Apart from the edges, normally we (prefer to) use weighting functions that have zero at the end point/curve/surfaces. This will finally make the above expression simpler, if $w$ is $N$-times derivable.

\begin{equation}
\int_{y_1}^{y_2} dy f(y) w(y) = \sum_{i=1 \ \textrm{or} \ 2}^{N} (-1)^{(i-1)}\left [ F^{(i)}(y) \frac{\partial^{(i-1)}}{\partial y^{(i-1)}}w(y)\right ]_{y_1}^{y_2}
\end{equation}


In [106]:
orderUtime=4
orderUspace=4
#m = 15 # the midpoint to evaluate the operator at
L₁ₗ = 1 # the leftmost point for x1 direction
L₁ᵣ = 1 # the rightmost point
Lₜₗ = 1 # the leftmost point for t direction
Lₜᵣ = 1 # the rightmost point


orderStructure=L₁ₗ+L₁ᵣ # number of nodes minus one

@variables dx₁ dt   
@variables Cᴾ[1:orderStructure+1] Cᴹ[1:orderStructure+1] Cᵘ[1:orderUspace+1,1:orderUtime+1] # These are the derivatives at the node point xₘ, tₙ
#@variables x¹ t




#@variables ξ¹[1:L₁ₗ+L₁ᵣ+1]  τ[1:Lₜₗ+Lₜᵣ+1] # These are the node points in the spatial and time domain
#@variables ξ¹ₘ[-L₁ₗ:L₁ᵣ] τₙ[-L₁ₗ:L₁ᵣ] ρₘ[-L₁ₗ:L₁ᵣ] μₘ[-L₁ₗ:L₁ᵣ]
ξ¹ₘ = Symbolics.variables(:ξ¹ₘ,-L₁ₗ:L₁ᵣ) # nodes in space (x₁ direction)
τₙ = Symbolics.variables(:τₙ,-Lₜₗ:Lₜᵣ) # nodes in time
ρₘ = Symbolics.variables(:ρₘ,-L₁ₗ:L₁ᵣ)
μₘ = Symbolics.variables(:μₘ,-L₁ₗ:L₁ᵣ)
uₘₙ= Symbolics.variables(:uₘₙ,-L₁ₗ:L₁ᵣ,-Lₜₗ:Lₜᵣ)
Aₘₙ= Symbolics.variables(:Aₘₙ,-L₁ₗ:L₁ᵣ,-Lₜₗ:Lₜᵣ)
A_GT98=Symbolics.variables(:A_GT98,-L₁ₗ:L₁ᵣ,-Lₜₗ:Lₜᵣ)
A_classic=Symbolics.variables(:A_classic,-L₁ₗ:L₁ᵣ,-Lₜₗ:Lₜᵣ)

ρ(dx₁) = sum(Cᴾ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
μ(dx₁) = sum(Cᴹ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
u(dx₁,dt) = sum(sum(Cᵘ[i,j] * dx₁^(i-1) * dt^(j-1) / factorial(BigInt(i-1)) / factorial(BigInt(j-1)) for i in 1:orderUspace+1) for j in 1:orderUtime+1)

u (generic function with 1 method)

## First step
We need to construct the Taylor expansion coefficients for $\rho$ and $\mu$ from nodes

In [107]:
# dummy X and T for X = x-xₘ T = t-tₘ
@variables X,T # (continuous) x and t from xₘ and tₙ
# define derivatives
∂ₜ=Differential(T)
∂ₓ=Differential(X)
∂ₜ²=∂ₜ^2
∂ₓ²=∂ₓ^2

Differential(X) ∘ Differential(X)

In [108]:
ρ(ξ¹ₘ[2]-ξ¹ₘ[1])

Cᴾ[1] + Cᴾ[2]*(ξ¹ₘ₀ - ξ¹ₘ₋₁) + (1//2)*Cᴾ[3]*((ξ¹ₘ₀ - ξ¹ₘ₋₁)^2)

If we use an arregular mesh:

In [109]:
# This should be used for an irregular mesh
eqsρ=[]
eqsμ=[]
eqsu=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=ξ¹ₘ[i¹]-ξ¹ₘ[L₁ₗ+1]
    eqsρ = push!(eqsρ, ρₘ[i¹]~ ρ(dx¹))
    eqsμ = push!(eqsμ, μₘ[i¹]~ μ(dx¹))
    #for iᵗ in 1:Lₜₗ+Lₜᵣ+1
    #    dt = τₙ[iᵗ]-τₙ[Lₜₗ+1]
    #    #eqsu = push!(eqsu, uₘₙ[i¹,iᵗ]~u(dx¹,dt))
    #end
end



If we use a regular mesh:


In [110]:
# Here we can also think about a regular mesh
@variables dx Δx dt Δt
eqsρ=[]
eqsμ=[]
eqsu=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    eqsρ = push!(eqsρ, ρₘ[i¹]~ ρ(dx¹))
    eqsμ = push!(eqsμ, μₘ[i¹]~ μ(dx¹))
    #for iᵗ in 1:Lₜₗ+Lₜᵣ+1
    #    dt = Δt*(iᵗ-Lₜₗ-1)
    #    eqsu = push!(eqsu, uₘₙ[i¹,iᵗ]~u(dx¹,dt))
    #end
end

In [111]:
newCᴾ=Symbolics.solve_for(eqsρ,Cᴾ)
newCᴹ=Symbolics.solve_for(eqsμ,Cᴹ) # if I do simplify here, it doesn't do a good job
#newCᵘ=reshape(Symbolics.solve_for(eqsu,Cᵘ),(L₁ₗ+L₁ᵣ+1,Lₜₗ+Lₜᵣ+1))
simplify(newCᴹ[2]) # maybe for the moment simplify should work only for an element
# this newCᴹ and newCᴾ are the derivatives to reconstruct polynomial expressions for continuous ρ and μ

newρ(dx₁) = sum(newCᴾ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
newμ(dx₁) = sum(newCᴹ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
#newu(dx₁,dt) = sum(sum(newCᵘ[i,j] * dx₁^(i-1) * dt^(j-1) / factorial(BigInt(i-1)) / factorial(BigInt(j-1)) for i in 1:orderUspace+1) for j in 1:orderUtime+1)

newμ (generic function with 1 method)

In [112]:
simplify(newμ(X))
#Symbolics.coeff(X^2+2*X,X)

(-(X^2)*μₘ₀ + (1//2)*(X^2)*μₘ₁ + (1//2)*(X^2)*μₘ₋₁ + (1//2)*X*Δx*μₘ₁ - (1//2)*X*Δx*μₘ₋₁ + (Δx^2)*μₘ₀) / (Δx^2)

In [113]:
simplify(u(X,T))

Cᵘ[1, 1] + Cᵘ[1, 2]*T + Cᵘ[2, 1]*X + (1//2)*Cᵘ[1, 3]*(T^2) + Cᵘ[2, 2]*T*X + (1//2)*Cᵘ[3, 1]*(X^2) + (1//6)*Cᵘ[1, 4]*(T^3) + (1//2)*Cᵘ[2, 3]*(T^2)*X + (1//2)*Cᵘ[3, 2]*T*(X^2) + (1//6)*Cᵘ[4, 1]*(X^3) + (1//24)*Cᵘ[1, 5]*(T^4) + (1//6)*Cᵘ[2, 4]*(T^3)*X + (1//4)*Cᵘ[3, 3]*(T^2)*(X^2) + (1//6)*Cᵘ[4, 2]*T*(X^3) + (1//24)*Cᵘ[5, 1]*(X^4) + (1//24)*Cᵘ[2, 5]*(T^4)*X + (1//12)*Cᵘ[3, 4]*(T^3)*(X^2) + (1//12)*Cᵘ[4, 3]*(T^2)*(X^3) + (1//24)*Cᵘ[5, 2]*T*(X^4) + (1//48)*Cᵘ[3, 5]*(T^4)*(X^2) + (1//36)*Cᵘ[4, 4]*(T^3)*(X^3) + (1//48)*Cᵘ[5, 3]*(T^2)*(X^4) + (1//144)*Cᵘ[4, 5]*(T^4)*(X^3) + (1//144)*Cᵘ[5, 4]*(T^3)*(X^4) + (1//576)*Cᵘ[5, 5]*(T^4)*(X^4)

# Second step: but in a homogeneous medium

In [114]:
# expression of acceleration:
@variables ρₒ μₒ
ρü=ρₒ* expand_derivatives(∂ₜ²(u(X,T)))
ρü=simplify(ρü;expand=true,threaded=false)


Cᵘ[1, 3]*ρₒ + Cᵘ[1, 4]*T*ρₒ + Cᵘ[2, 3]*X*ρₒ + (1//2)*Cᵘ[1, 5]*(T^2)*ρₒ + Cᵘ[2, 4]*T*X*ρₒ + (1//2)*Cᵘ[3, 3]*(X^2)*ρₒ + (1//2)*Cᵘ[2, 5]*(T^2)*X*ρₒ + (1//2)*Cᵘ[3, 4]*T*(X^2)*ρₒ + (1//6)*Cᵘ[4, 3]*(X^3)*ρₒ + (1//4)*Cᵘ[3, 5]*(T^2)*(X^2)*ρₒ + (1//6)*Cᵘ[4, 4]*T*(X^3)*ρₒ + (1//24)*Cᵘ[5, 3]*(X^4)*ρₒ + (1//12)*Cᵘ[4, 5]*(T^2)*(X^3)*ρₒ + (1//24)*Cᵘ[5, 4]*T*(X^4)*ρₒ + (1//48)*Cᵘ[5, 5]*(T^2)*(X^4)*ρₒ

In [115]:
# expression of (μu′)′

μu′ = simplify(μₒ*expand_derivatives(∂ₓ(u(X,T)));expand=true)
kinematics= simplify(expand_derivatives(∂ₓ(μu′));expand=true)

Cᵘ[3, 1]*μₒ + Cᵘ[3, 2]*T*μₒ + Cᵘ[4, 1]*X*μₒ + (1//2)*Cᵘ[3, 3]*(T^2)*μₒ + Cᵘ[4, 2]*T*X*μₒ + (1//2)*Cᵘ[5, 1]*(X^2)*μₒ + (1//6)*Cᵘ[3, 4]*(T^3)*μₒ + (1//2)*Cᵘ[4, 3]*(T^2)*X*μₒ + (1//2)*Cᵘ[5, 2]*T*(X^2)*μₒ + (1//24)*Cᵘ[3, 5]*(T^4)*μₒ + (1//6)*Cᵘ[4, 4]*(T^3)*X*μₒ + (1//4)*Cᵘ[5, 3]*(T^2)*(X^2)*μₒ + (1//24)*Cᵘ[4, 5]*(T^4)*X*μₒ + (1//12)*Cᵘ[5, 4]*(T^3)*(X^2)*μₒ + (1//48)*Cᵘ[5, 5]*(T^4)*(X^2)*μₒ

In [116]:
# here we give the integrand
integrand=ρü-kinematics

Cᵘ[1, 3]*ρₒ - Cᵘ[3, 1]*μₒ + Cᵘ[1, 4]*T*ρₒ + Cᵘ[2, 3]*X*ρₒ - Cᵘ[3, 2]*T*μₒ - Cᵘ[4, 1]*X*μₒ + (1//2)*Cᵘ[1, 5]*(T^2)*ρₒ + Cᵘ[2, 4]*T*X*ρₒ - (1//2)*Cᵘ[3, 3]*(T^2)*μₒ + (1//2)*Cᵘ[3, 3]*(X^2)*ρₒ - Cᵘ[4, 2]*T*X*μₒ - (1//2)*Cᵘ[5, 1]*(X^2)*μₒ + (1//2)*Cᵘ[2, 5]*(T^2)*X*ρₒ - (1//6)*Cᵘ[3, 4]*(T^3)*μₒ + (1//2)*Cᵘ[3, 4]*T*(X^2)*ρₒ - (1//2)*Cᵘ[4, 3]*(T^2)*X*μₒ + (1//6)*Cᵘ[4, 3]*(X^3)*ρₒ - (1//2)*Cᵘ[5, 2]*T*(X^2)*μₒ - (1//24)*Cᵘ[3, 5]*(T^4)*μₒ + (1//4)*Cᵘ[3, 5]*(T^2)*(X^2)*ρₒ - (1//6)*Cᵘ[4, 4]*(T^3)*X*μₒ + (1//6)*Cᵘ[4, 4]*T*(X^3)*ρₒ - (1//4)*Cᵘ[5, 3]*(T^2)*(X^2)*μₒ + (1//24)*Cᵘ[5, 3]*(X^4)*ρₒ - (1//24)*Cᵘ[4, 5]*(T^4)*X*μₒ + (1//12)*Cᵘ[4, 5]*(T^2)*(X^3)*ρₒ - (1//12)*Cᵘ[5, 4]*(T^3)*(X^2)*μₒ + (1//24)*Cᵘ[5, 4]*T*(X^4)*ρₒ - (1//48)*Cᵘ[5, 5]*(T^4)*(X^2)*μₒ + (1//48)*Cᵘ[5, 5]*(T^2)*(X^4)*ρₒ

In [117]:
Symbolics.coeff(integrand,X)

Cᵘ[2, 3]*ρₒ - Cᵘ[4, 1]*μₒ + Cᵘ[2, 4]*T*ρₒ - Cᵘ[4, 2]*T*μₒ + (1//2)*Cᵘ[2, 5]*(T^2)*ρₒ - (1//2)*Cᵘ[4, 3]*(T^2)*μₒ - (1//6)*Cᵘ[4, 4]*(T^3)*μₒ - (1//24)*Cᵘ[4, 5]*(T^4)*μₒ

In [118]:
f1_space=integrateTaylorPolynomials(integrand,X)


Cᵘ[1, 3]*X*ρₒ - Cᵘ[3, 1]*X*μₒ + Cᵘ[1, 4]*T*X*ρₒ + (1//2)*Cᵘ[2, 3]*(X^2)*ρₒ - Cᵘ[3, 2]*T*X*μₒ - (1//2)*Cᵘ[4, 1]*(X^2)*μₒ + (1//2)*Cᵘ[1, 5]*(T^2)*X*ρₒ + (1//2)*Cᵘ[2, 4]*T*(X^2)*ρₒ - (1//2)*Cᵘ[3, 3]*(T^2)*X*μₒ + (1//6)*Cᵘ[3, 3]*(X^3)*ρₒ - (1//2)*Cᵘ[4, 2]*T*(X^2)*μₒ - (1//6)*Cᵘ[5, 1]*(X^3)*μₒ + (1//4)*Cᵘ[2, 5]*(T^2)*(X^2)*ρₒ - (1//6)*Cᵘ[3, 4]*(T^3)*X*μₒ + (1//6)*Cᵘ[3, 4]*T*(X^3)*ρₒ - (1//4)*Cᵘ[4, 3]*(T^2)*(X^2)*μₒ + (1//24)*Cᵘ[4, 3]*(X^4)*ρₒ - (1//6)*Cᵘ[5, 2]*T*(X^3)*μₒ - (1//24)*Cᵘ[3, 5]*(T^4)*X*μₒ + (1//12)*Cᵘ[3, 5]*(T^2)*(X^3)*ρₒ - (1//12)*Cᵘ[4, 4]*(T^3)*(X^2)*μₒ + (1//24)*Cᵘ[4, 4]*T*(X^4)*ρₒ - (1//12)*Cᵘ[5, 3]*(T^2)*(X^3)*μₒ + (1//120)*Cᵘ[5, 3]*(X^5)*ρₒ - (1//48)*Cᵘ[4, 5]*(T^4)*(X^2)*μₒ + (1//48)*Cᵘ[4, 5]*(T^2)*(X^4)*ρₒ - (1//36)*Cᵘ[5, 4]*(T^3)*(X^3)*μₒ + (1//120)*Cᵘ[5, 4]*T*(X^5)*ρₒ - (1//144)*Cᵘ[5, 5]*(T^4)*(X^3)*μₒ + (1//240)*Cᵘ[5, 5]*(T^2)*(X^5)*ρₒ

In [119]:

f2_space=integrateTaylorPolynomials(f1_space,X)

(1//2)*Cᵘ[1, 3]*(X^2)*ρₒ - (1//2)*Cᵘ[3, 1]*(X^2)*μₒ + (1//2)*Cᵘ[1, 4]*T*(X^2)*ρₒ + (1//6)*Cᵘ[2, 3]*(X^3)*ρₒ - (1//2)*Cᵘ[3, 2]*T*(X^2)*μₒ - (1//6)*Cᵘ[4, 1]*(X^3)*μₒ + (1//4)*Cᵘ[1, 5]*(T^2)*(X^2)*ρₒ + (1//6)*Cᵘ[2, 4]*T*(X^3)*ρₒ - (1//4)*Cᵘ[3, 3]*(T^2)*(X^2)*μₒ + (1//24)*Cᵘ[3, 3]*(X^4)*ρₒ - (1//6)*Cᵘ[4, 2]*T*(X^3)*μₒ - (1//24)*Cᵘ[5, 1]*(X^4)*μₒ + (1//12)*Cᵘ[2, 5]*(T^2)*(X^3)*ρₒ - (1//12)*Cᵘ[3, 4]*(T^3)*(X^2)*μₒ + (1//24)*Cᵘ[3, 4]*T*(X^4)*ρₒ - (1//12)*Cᵘ[4, 3]*(T^2)*(X^3)*μₒ + (1//120)*Cᵘ[4, 3]*(X^5)*ρₒ - (1//24)*Cᵘ[5, 2]*T*(X^4)*μₒ - (1//48)*Cᵘ[3, 5]*(T^4)*(X^2)*μₒ + (1//48)*Cᵘ[3, 5]*(T^2)*(X^4)*ρₒ - (1//36)*Cᵘ[4, 4]*(T^3)*(X^3)*μₒ + (1//120)*Cᵘ[4, 4]*T*(X^5)*ρₒ - (1//48)*Cᵘ[5, 3]*(T^2)*(X^4)*μₒ + (1//720)*Cᵘ[5, 3]*(X^6)*ρₒ - (1//144)*Cᵘ[4, 5]*(T^4)*(X^3)*μₒ + (1//240)*Cᵘ[4, 5]*(T^2)*(X^5)*ρₒ - (1//144)*Cᵘ[5, 4]*(T^3)*(X^4)*μₒ + (1//720)*Cᵘ[5, 4]*T*(X^6)*ρₒ - (1//576)*Cᵘ[5, 5]*(T^4)*(X^4)*μₒ + (1//1440)*Cᵘ[5, 5]*(T^2)*(X^6)*ρₒ

In [120]:
# this is F^{[2]} 
f2_space_right=substitute(f2_space,Dict([X=>Δx]))
f2_space_left=substitute(f2_space,Dict([X=>(-Δx)]))

(1//2)*Cᵘ[1, 3]*(Δx^2)*ρₒ - (1//2)*Cᵘ[3, 1]*(Δx^2)*μₒ + (1//2)*Cᵘ[1, 4]*T*(Δx^2)*ρₒ - (1//6)*Cᵘ[2, 3]*(Δx^3)*ρₒ - (1//2)*Cᵘ[3, 2]*T*(Δx^2)*μₒ + (1//6)*Cᵘ[4, 1]*(Δx^3)*μₒ + (1//4)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₒ - (1//6)*Cᵘ[2, 4]*T*(Δx^3)*ρₒ - (1//4)*Cᵘ[3, 3]*(T^2)*(Δx^2)*μₒ + (1//24)*Cᵘ[3, 3]*(Δx^4)*ρₒ + (1//6)*Cᵘ[4, 2]*T*(Δx^3)*μₒ - (1//24)*Cᵘ[5, 1]*(Δx^4)*μₒ - (1//12)*Cᵘ[2, 5]*(T^2)*(Δx^3)*ρₒ - (1//12)*Cᵘ[3, 4]*(T^3)*(Δx^2)*μₒ + (1//24)*Cᵘ[3, 4]*T*(Δx^4)*ρₒ + (1//12)*Cᵘ[4, 3]*(T^2)*(Δx^3)*μₒ - (1//120)*Cᵘ[4, 3]*(Δx^5)*ρₒ - (1//24)*Cᵘ[5, 2]*T*(Δx^4)*μₒ - (1//48)*Cᵘ[3, 5]*(T^4)*(Δx^2)*μₒ + (1//48)*Cᵘ[3, 5]*(T^2)*(Δx^4)*ρₒ + (1//36)*Cᵘ[4, 4]*(T^3)*(Δx^3)*μₒ - (1//120)*Cᵘ[4, 4]*T*(Δx^5)*ρₒ - (1//48)*Cᵘ[5, 3]*(T^2)*(Δx^4)*μₒ + (1//720)*Cᵘ[5, 3]*(Δx^6)*ρₒ + (1//144)*Cᵘ[4, 5]*(T^4)*(Δx^3)*μₒ - (1//240)*Cᵘ[4, 5]*(T^2)*(Δx^5)*ρₒ - (1//144)*Cᵘ[5, 4]*(T^3)*(Δx^4)*μₒ + (1//720)*Cᵘ[5, 4]*T*(Δx^6)*ρₒ - (1//576)*Cᵘ[5, 5]*(T^4)*(Δx^4)*μₒ + (1//1440)*Cᵘ[5, 5]*(T^2)*(Δx^6)*ρₒ

In [121]:
f2_space_zero=substitute(f2_space,Dict([X=>0]))

0

Here we want to evaluate:
\begin{equation}
-\left [ F^{[2]}(x) \phi^{(1)}(x) \right ]^{\Delta x}_{-\Delta  x}
\end{equation}
where we have 
\begin{align}
\phi^{(1)}(\Delta x) &=& -\frac{1}{\Delta x}\\
\phi^{(1)}(-\Delta x) &=& \frac{1}{\Delta x}
\end{align}


In [122]:
ϕ_deriv_right=-1//Δx
ϕ_deriv_left = 1//Δx
desired_value_in_space=-(ϕ_deriv_right*f2_space_right-ϕ_deriv_left*f2_space_left)

((1//2)*Cᵘ[1, 3]*(Δx^2)*ρₒ - (1//2)*Cᵘ[3, 1]*(Δx^2)*μₒ + (1//2)*Cᵘ[1, 4]*T*(Δx^2)*ρₒ + (1//6)*Cᵘ[2, 3]*(Δx^3)*ρₒ - (1//2)*Cᵘ[3, 2]*T*(Δx^2)*μₒ - (1//6)*Cᵘ[4, 1]*(Δx^3)*μₒ + (1//4)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₒ + (1//6)*Cᵘ[2, 4]*T*(Δx^3)*ρₒ - (1//4)*Cᵘ[3, 3]*(T^2)*(Δx^2)*μₒ + (1//24)*Cᵘ[3, 3]*(Δx^4)*ρₒ - (1//6)*Cᵘ[4, 2]*T*(Δx^3)*μₒ - (1//24)*Cᵘ[5, 1]*(Δx^4)*μₒ + (1//12)*Cᵘ[2, 5]*(T^2)*(Δx^3)*ρₒ - (1//12)*Cᵘ[3, 4]*(T^3)*(Δx^2)*μₒ + (1//24)*Cᵘ[3, 4]*T*(Δx^4)*ρₒ - (1//12)*Cᵘ[4, 3]*(T^2)*(Δx^3)*μₒ + (1//120)*Cᵘ[4, 3]*(Δx^5)*ρₒ - (1//24)*Cᵘ[5, 2]*T*(Δx^4)*μₒ - (1//48)*Cᵘ[3, 5]*(T^4)*(Δx^2)*μₒ + (1//48)*Cᵘ[3, 5]*(T^2)*(Δx^4)*ρₒ - (1//36)*Cᵘ[4, 4]*(T^3)*(Δx^3)*μₒ + (1//120)*Cᵘ[4, 4]*T*(Δx^5)*ρₒ - (1//48)*Cᵘ[5, 3]*(T^2)*(Δx^4)*μₒ + (1//720)*Cᵘ[5, 3]*(Δx^6)*ρₒ - (1//144)*Cᵘ[4, 5]*(T^4)*(Δx^3)*μₒ + (1//240)*Cᵘ[4, 5]*(T^2)*(Δx^5)*ρₒ - (1//144)*Cᵘ[5, 4]*(T^3)*(Δx^4)*μₒ + (1//720)*Cᵘ[5, 4]*T*(Δx^6)*ρₒ - (1//576)*Cᵘ[5, 5]*(T^4)*(Δx^4)*μₒ + (1//1440)*Cᵘ[5, 5]*(T^2)*(Δx^6)*ρₒ) / Δx + ((1//2)*Cᵘ[1, 3]*(Δx^2)

In [123]:
# now time integral

f1_time=integrateTaylorPolynomials(desired_value_in_space,T)
f2_time=integrateTaylorPolynomials(f1_time,T)


(1//2)*Cᵘ[1, 3]*(T^2)*Δx*ρₒ - (1//2)*Cᵘ[3, 1]*(T^2)*Δx*μₒ + (1//6)*Cᵘ[1, 4]*(T^3)*Δx*ρₒ - (1//6)*Cᵘ[3, 2]*(T^3)*Δx*μₒ + (1//24)*Cᵘ[1, 5]*(T^4)*Δx*ρₒ - (1//24)*Cᵘ[3, 3]*(T^4)*Δx*μₒ + (1//24)*Cᵘ[3, 3]*(T^2)*(Δx^3)*ρₒ - (1//24)*Cᵘ[5, 1]*(T^2)*(Δx^3)*μₒ - (1//120)*Cᵘ[3, 4]*(T^5)*Δx*μₒ + (1//72)*Cᵘ[3, 4]*(T^3)*(Δx^3)*ρₒ - (1//72)*Cᵘ[5, 2]*(T^3)*(Δx^3)*μₒ - (1//720)*Cᵘ[3, 5]*(T^6)*Δx*μₒ + (1//288)*Cᵘ[3, 5]*(T^4)*(Δx^3)*ρₒ - (1//288)*Cᵘ[5, 3]*(T^4)*(Δx^3)*μₒ + (1//720)*Cᵘ[5, 3]*(T^2)*(Δx^5)*ρₒ - (1//1440)*Cᵘ[5, 4]*(T^5)*(Δx^3)*μₒ + (1//2160)*Cᵘ[5, 4]*(T^3)*(Δx^5)*ρₒ - (1//8640)*Cᵘ[5, 5]*(T^6)*(Δx^3)*μₒ + (1//8640)*Cᵘ[5, 5]*(T^4)*(Δx^5)*ρₒ

In [124]:
f2_time_right=substitute(f2_time,Dict([T=>Δt]))
f2_time_left=substitute(f2_time,Dict([T=>(-Δt)]))

(1//2)*Cᵘ[1, 3]*(Δt^2)*Δx*ρₒ - (1//2)*Cᵘ[3, 1]*(Δt^2)*Δx*μₒ - (1//6)*Cᵘ[1, 4]*(Δt^3)*Δx*ρₒ + (1//6)*Cᵘ[3, 2]*(Δt^3)*Δx*μₒ + (1//24)*Cᵘ[1, 5]*(Δt^4)*Δx*ρₒ - (1//24)*Cᵘ[3, 3]*(Δt^4)*Δx*μₒ + (1//24)*Cᵘ[3, 3]*(Δt^2)*(Δx^3)*ρₒ - (1//24)*Cᵘ[5, 1]*(Δt^2)*(Δx^3)*μₒ + (1//120)*Cᵘ[3, 4]*(Δt^5)*Δx*μₒ - (1//72)*Cᵘ[3, 4]*(Δt^3)*(Δx^3)*ρₒ + (1//72)*Cᵘ[5, 2]*(Δt^3)*(Δx^3)*μₒ - (1//720)*Cᵘ[3, 5]*(Δt^6)*Δx*μₒ + (1//288)*Cᵘ[3, 5]*(Δt^4)*(Δx^3)*ρₒ - (1//288)*Cᵘ[5, 3]*(Δt^4)*(Δx^3)*μₒ + (1//720)*Cᵘ[5, 3]*(Δt^2)*(Δx^5)*ρₒ + (1//1440)*Cᵘ[5, 4]*(Δt^5)*(Δx^3)*μₒ - (1//2160)*Cᵘ[5, 4]*(Δt^3)*(Δx^5)*ρₒ - (1//8640)*Cᵘ[5, 5]*(Δt^6)*(Δx^3)*μₒ + (1//8640)*Cᵘ[5, 5]*(Δt^4)*(Δx^5)*ρₒ

In [125]:
ψ_deriv_right=-1//Δt
ψ_deriv_left = 1//Δt
desired_value_in_time=-(ψ_deriv_right*f2_time_right-ψ_deriv_left*f2_time_left)
desired_value_in_time=simplify(desired_value_in_time;expand=true)

Cᵘ[1, 3]*Δt*Δx*ρₒ - Cᵘ[3, 1]*Δt*Δx*μₒ + (1//12)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₒ - (1//12)*Cᵘ[3, 3]*(Δt^3)*Δx*μₒ + (1//12)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₒ - (1//12)*Cᵘ[5, 1]*Δt*(Δx^3)*μₒ - (1//360)*Cᵘ[3, 5]*(Δt^5)*Δx*μₒ + (1//144)*Cᵘ[3, 5]*(Δt^3)*(Δx^3)*ρₒ - (1//144)*Cᵘ[5, 3]*(Δt^3)*(Δx^3)*μₒ + (1//360)*Cᵘ[5, 3]*Δt*(Δx^5)*ρₒ - (1//4320)*Cᵘ[5, 5]*(Δt^5)*(Δx^3)*μₒ + (1//4320)*Cᵘ[5, 5]*(Δt^3)*(Δx^5)*ρₒ

\begin{equation}
 \sum_{m,n} A_{mn} u_{mn}
 \end{equation}

In [126]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=Aₘₙ[i¹,iᵗ]*u(dx¹,dt)
    end
end
difference_between_matrix_strongform=simplify(innerProduct-desired_value_in_time;expand=true)
Symbolics.coeff(difference_between_matrix_strongform,Cᵘ[1,3])

(1//2)*Aₘₙ₀ˏ₁*(Δt^2) + (1//2)*Aₘₙ₀ˏ₋₁*(Δt^2) + (1//2)*Aₘₙ₁ˏ₁*(Δt^2) + (1//2)*Aₘₙ₁ˏ₋₁*(Δt^2) + (1//2)*Aₘₙ₋₁ˏ₁*(Δt^2) + (1//2)*Aₘₙ₋₁ˏ₋₁*(Δt^2) - Δt*Δx*ρₒ

In [ ]:
innerProduct

In [127]:
hierarchical_equations=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1   
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1  
        push!(hierarchical_equations,Symbolics.coeff(difference_between_matrix_strongform,Cᵘ[i¹,iᵗ])~0)
    end
end
newA=reshape(Symbolics.solve_for(hierarchical_equations,Aₘₙ),(L₁ₗ+L₁ᵣ+1,Lₜₗ+Lₜᵣ+1))
simplify(newA[1,3])

((Δt^2)*μₒ - (Δx^2)*ρₒ) / (-12Δt*Δx)

In [128]:

for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(hierarchical_equations[(i¹-1)*(Lₜₗ+Lₜᵣ+1)+iᵗ]))
    end
end

Aₘₙ₀ˏ₀ + Aₘₙ₀ˏ₁ + Aₘₙ₀ˏ₋₁ + Aₘₙ₁ˏ₀ + Aₘₙ₁ˏ₁ + Aₘₙ₁ˏ₋₁ + Aₘₙ₋₁ˏ₀ + Aₘₙ₋₁ˏ₁ + Aₘₙ₋₁ˏ₋₁ ~ 0

(Aₘₙ₀ˏ₁ + Aₘₙ₁ˏ₁ + Aₘₙ₋₁ˏ₁)*Δt - Aₘₙ₀ˏ₋₁*Δt - Aₘₙ₁ˏ₋₁*Δt - Aₘₙ₋₁ˏ₋₁*Δt ~ 0

(1//2)*Aₘₙ₀ˏ₁*(Δt^2) + (1//2)*Aₘₙ₀ˏ₋₁*(Δt^2) + (1//2)*Aₘₙ₁ˏ₁*(Δt^2) + (1//2)*Aₘₙ₁ˏ₋₁*(Δt^2) + (1//2)*Aₘₙ₋₁ˏ₁*(Δt^2) + (1//2)*Aₘₙ₋₁ˏ₋₁*(Δt^2) - Δt*Δx*ρₒ ~ 0

(Aₘₙ₁ˏ₀ + Aₘₙ₁ˏ₁ + Aₘₙ₁ˏ₋₁)*Δx - Aₘₙ₋₁ˏ₀*Δx - Aₘₙ₋₁ˏ₁*Δx - Aₘₙ₋₁ˏ₋₁*Δx ~ 0

(Aₘₙ₁ˏ₁ + Aₘₙ₋₁ˏ₋₁)*Δt*Δx - Aₘₙ₁ˏ₋₁*Δt*Δx - Aₘₙ₋₁ˏ₁*Δt*Δx ~ 0

(1//2)*Aₘₙ₁ˏ₁*(Δt^2)*Δx + (1//2)*Aₘₙ₁ˏ₋₁*(Δt^2)*Δx - (1//2)*Aₘₙ₋₁ˏ₁*(Δt^2)*Δx - (1//2)*Aₘₙ₋₁ˏ₋₁*(Δt^2)*Δx ~ 0

(1//2)*Aₘₙ₁ˏ₀*(Δx^2) + (1//2)*Aₘₙ₁ˏ₁*(Δx^2) + (1//2)*Aₘₙ₁ˏ₋₁*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₀*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₁*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₋₁*(Δx^2) + Δt*Δx*μₒ ~ 0

(1//2)*Aₘₙ₁ˏ₁*Δt*(Δx^2) - (1//2)*Aₘₙ₁ˏ₋₁*Δt*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₁*Δt*(Δx^2) - (1//2)*Aₘₙ₋₁ˏ₋₁*Δt*(Δx^2) ~ 0

(1//4)*Aₘₙ₁ˏ₁*(Δt^2)*(Δx^2) + (1//4)*Aₘₙ₁ˏ₋₁*(Δt^2)*(Δx^2) + (1//4)*Aₘₙ₋₁ˏ₁*(Δt^2)*(Δx^2) + (1//4)*Aₘₙ₋₁ˏ₋₁*(Δt^2)*(Δx^2) + (1//12)*(Δt^3)*Δx*μₒ - (1//12)*Δt*(Δx^3)*ρₒ ~ 0

In [129]:
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(newA[i¹,iᵗ];expand=true))
    end
end

(-(1//12)*(Δt^2)*μₒ + (1//12)*(Δx^2)*ρₒ) / (Δt*Δx)

((5//1)*(Δt^2)*μₒ + (Δx^2)*ρₒ) / (-6Δt*Δx)

((Δt^2)*μₒ - (Δx^2)*ρₒ) / (-12Δt*Δx)

(-(Δt^2)*μₒ - (5//1)*(Δx^2)*ρₒ) / (-6Δt*Δx)

(-(5//1)*(Δt^2)*μₒ + (5//1)*(Δx^2)*ρₒ) / (-3Δt*Δx)

(-(Δt^2)*μₒ - (5//1)*(Δx^2)*ρₒ) / (-6Δt*Δx)

((Δt^2)*μₒ - (Δx^2)*ρₒ) / (-12Δt*Δx)

((5//6)*(Δt^2)*μₒ + (1//6)*(Δx^2)*ρₒ) / (-Δt*Δx)

((1//12)*(Δt^2)*μₒ - (1//12)*(Δx^2)*ρₒ) / (-Δt*Δx)

# check errors

In [130]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=newA[i¹,iᵗ]*u(dx¹,dt)
    end
end
difference=simplify((innerProduct-desired_value_in_time);expand=true)
display(difference)
truncated_difference=ignoreHigherOrderTerms(difference,Δx,3)
truncated_difference=ignoreHigherOrderTerms(truncated_difference,Δt,3)
display(truncated_difference)

(1//2880)*(-(12//1)*Cᵘ[3, 5]*(Δt^5)*Δx*μₒ + (12//1)*Cᵘ[5, 3]*Δt*(Δx^5)*ρₒ - Cᵘ[5, 5]*(Δt^5)*(Δx^3)*μₒ + Cᵘ[5, 5]*(Δt^3)*(Δx^5)*ρₒ)

0

In [131]:

A_classic.=0
A_classic[2,1]=ρₒ//Δt^2
A_classic[2,2]=-2*ρₒ//Δt^2 + 2*μₒ//(Δx^2)
A_classic[2,3]=ρₒ//Δt^2

A_classic[1,2] = -μₒ//(Δx^2)
A_classic[3,2] = -μₒ//(Δx^2)

A_classic=A_classic*Δx*Δt

for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(A_classic[i¹,iᵗ];expand=true))
    end
end

innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=A_classic[i¹,iᵗ]*u(dx¹,dt)
        display(u(dx¹,dt))
    end
end
differenceC=simplify(innerProduct-desired_value_in_time,expand=true)
display(differenceC)
truncated_differenceC=ignoreHigherOrderTerms(differenceC,Δx,3)
truncated_differenceC=ignoreHigherOrderTerms(truncated_differenceC,Δt,3)
display(truncated_differenceC)

0

(-Δt*μₒ) / Δx

0

(Δx*ρₒ) / Δt

(2(Δt^2)*μₒ - 2(Δx^2)*ρₒ) / (Δt*Δx)

...

Cᵘ[1, 1] - Cᵘ[1, 2]*Δt + Cᵘ[2, 1]*Δx + (1//2)*Cᵘ[1, 3]*(Δt^2) - Cᵘ[2, 2]*Δt*Δx + (1//2)*Cᵘ[3, 1]*(Δx^2) - (1//6)*Cᵘ[1, 4]*(Δt^3) + (1//2)*Cᵘ[2, 3]*(Δt^2)*Δx - (1//2)*Cᵘ[3, 2]*Δt*(Δx^2) + (1//6)*Cᵘ[4, 1]*(Δx^3) + (1//24)*Cᵘ[1, 5]*(Δt^4) - (1//6)*Cᵘ[2, 4]*(Δt^3)*Δx + (1//4)*Cᵘ[3, 3]*(Δt^2)*(Δx^2) - (1//6)*Cᵘ[4, 2]*Δt*(Δx^3) + (1//24)*Cᵘ[5, 1]*(Δx^4) + (1//24)*Cᵘ[2, 5]*(Δt^4)*Δx - (1//12)*Cᵘ[3, 4]*(Δt^3)*(Δx^2) + (1//12)*Cᵘ[4, 3]*(Δt^2)*(Δx^3) - (1//24)*Cᵘ[5, 2]*Δt*(Δx^4) + (1//48)*Cᵘ[3, 5]*(Δt^4)*(Δx^2) - (1//36)*Cᵘ[4, 4]*(Δt^3)*(Δx^3) + (1//48)*Cᵘ[5, 3]*(Δt^2)*(Δx^4) + (1//144)*Cᵘ[4, 5]*(Δt^4)*(Δx^3) - (1//144)*Cᵘ[5, 4]*(Δt^3)*(Δx^4) + (1//576)*Cᵘ[5, 5]*(Δt^4)*(Δx^4)

Cᵘ[1, 1] + Cᵘ[2, 1]*Δx + (1//2)*Cᵘ[3, 1]*(Δx^2) + (1//6)*Cᵘ[4, 1]*(Δx^3) + (1//24)*Cᵘ[5, 1]*(Δx^4)

Cᵘ[1, 1] + Cᵘ[1, 2]*Δt + Cᵘ[2, 1]*Δx + (1//2)*Cᵘ[1, 3]*(Δt^2) + Cᵘ[2, 2]*Δt*Δx + (1//2)*Cᵘ[3, 1]*(Δx^2) + (1//6)*Cᵘ[1, 4]*(Δt^3) + (1//2)*Cᵘ[2, 3]*(Δt^2)*Δx + (1//2)*Cᵘ[3, 2]*Δt*(Δx^2) + (1//6)*Cᵘ[4, 1]*(Δx^3) + (1//24)*Cᵘ[1, 5]*(Δt^4) + (1//6)*Cᵘ[2, 4]*(Δt^3)*Δx + (1//4)*Cᵘ[3, 3]*(Δt^2)*(Δx^2) + (1//6)*Cᵘ[4, 2]*Δt*(Δx^3) + (1//24)*Cᵘ[5, 1]*(Δx^4) + (1//24)*Cᵘ[2, 5]*(Δt^4)*Δx + (1//12)*Cᵘ[3, 4]*(Δt^3)*(Δx^2) + (1//12)*Cᵘ[4, 3]*(Δt^2)*(Δx^3) + (1//24)*Cᵘ[5, 2]*Δt*(Δx^4) + (1//48)*Cᵘ[3, 5]*(Δt^4)*(Δx^2) + (1//36)*Cᵘ[4, 4]*(Δt^3)*(Δx^3) + (1//48)*Cᵘ[5, 3]*(Δt^2)*(Δx^4) + (1//144)*Cᵘ[4, 5]*(Δt^4)*(Δx^3) + (1//144)*Cᵘ[5, 4]*(Δt^3)*(Δx^4) + (1//576)*Cᵘ[5, 5]*(Δt^4)*(Δx^4)

(1//12)*Cᵘ[3, 3]*(Δt^3)*Δx*μₒ - (1//12)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₒ + (1//360)*Cᵘ[3, 5]*(Δt^5)*Δx*μₒ - (1//144)*Cᵘ[3, 5]*(Δt^3)*(Δx^3)*ρₒ + (1//144)*Cᵘ[5, 3]*(Δt^3)*(Δx^3)*μₒ - (1//360)*Cᵘ[5, 3]*Δt*(Δx^5)*ρₒ + (1//4320)*Cᵘ[5, 5]*(Δt^5)*(Δx^3)*μₒ - (1//4320)*Cᵘ[5, 5]*(Δt^3)*(Δx^5)*ρₒ

(1//12)*Cᵘ[3, 3]*(Δt^3)*Δx*μₒ - (1//12)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₒ - (1//144)*Cᵘ[3, 5]*(Δt^3)*(Δx^3)*ρₒ + (1//144)*Cᵘ[5, 3]*(Δt^3)*(Δx^3)*μₒ

Don't be fooled by this bizarre denominators in $A_{mn}$. $A \mathbf{u} = \mathbf{F}$ with $\mathbf{u}$ discretised displacements whereas $\mathbf{F}$ is $\int dt \int dx f(x,t) \phi(x) \psi(t)$ which has $\Delta x \Delta x$ factor.

# Challenge : heterogeneous medium


In [79]:
display(newρ(X)|>simplify)
display(newμ(X)|>simplify)

(-(X^2)*ρₘ₀ + (1//2)*(X^2)*ρₘ₁ + (1//2)*(X^2)*ρₘ₋₁ + (1//2)*X*Δx*ρₘ₁ - (1//2)*X*Δx*ρₘ₋₁ + (Δx^2)*ρₘ₀) / (Δx^2)

(-(X^2)*μₘ₀ + (1//2)*(X^2)*μₘ₁ + (1//2)*(X^2)*μₘ₋₁ + (1//2)*X*Δx*μₘ₁ - (1//2)*X*Δx*μₘ₋₁ + (Δx^2)*μₘ₀) / (Δx^2)

In [ ]:
newρ(X)= ρₘ[2]+X/2/Δx*(ρₘ[3]-ρₘ[1])
newμ(X)=μₘ[2]+X/2/Δx*(μₘ[3]-μₘ[1])


In [80]:
# expression of acceleration:

ρü=newρ(X)* expand_derivatives(∂ₜ²(u(X,T)))
ρü=simplify(ρü;expand=true)


(-Cᵘ[1, 3]*(X^2)*ρₘ₀ + (1//2)*Cᵘ[1, 3]*(X^2)*ρₘ₁ + (1//2)*Cᵘ[1, 3]*(X^2)*ρₘ₋₁ + (1//2)*Cᵘ[1, 3]*X*Δx*ρₘ₁ - (1//2)*Cᵘ[1, 3]*X*Δx*ρₘ₋₁ + Cᵘ[1, 3]*(Δx^2)*ρₘ₀ - Cᵘ[1, 4]*T*(X^2)*ρₘ₀ + (1//2)*Cᵘ[1, 4]*T*(X^2)*ρₘ₁ + (1//2)*Cᵘ[1, 4]*T*(X^2)*ρₘ₋₁ + (1//2)*Cᵘ[1, 4]*T*X*Δx*ρₘ₁ - (1//2)*Cᵘ[1, 4]*T*X*Δx*ρₘ₋₁ + Cᵘ[1, 4]*T*(Δx^2)*ρₘ₀ - Cᵘ[2, 3]*(X^3)*ρₘ₀ + (1//2)*Cᵘ[2, 3]*(X^3)*ρₘ₁ + (1//2)*Cᵘ[2, 3]*(X^3)*ρₘ₋₁ + (1//2)*Cᵘ[2, 3]*(X^2)*Δx*ρₘ₁ - (1//2)*Cᵘ[2, 3]*(X^2)*Δx*ρₘ₋₁ + Cᵘ[2, 3]*X*(Δx^2)*ρₘ₀ - (1//2)*Cᵘ[1, 5]*(T^2)*(X^2)*ρₘ₀ + (1//4)*Cᵘ[1, 5]*(T^2)*(X^2)*ρₘ₁ + (1//4)*Cᵘ[1, 5]*(T^2)*(X^2)*ρₘ₋₁ + (1//4)*Cᵘ[1, 5]*(T^2)*X*Δx*ρₘ₁ - (1//4)*Cᵘ[1, 5]*(T^2)*X*Δx*ρₘ₋₁ + (1//2)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₀ - Cᵘ[2, 4]*T*(X^3)*ρₘ₀ + (1//2)*Cᵘ[2, 4]*T*(X^3)*ρₘ₁ + (1//2)*Cᵘ[2, 4]*T*(X^3)*ρₘ₋₁ + (1//2)*Cᵘ[2, 4]*T*(X^2)*Δx*ρₘ₁ - (1//2)*Cᵘ[2, 4]*T*(X^2)*Δx*ρₘ₋₁ + Cᵘ[2, 4]*T*X*(Δx^2)*ρₘ₀ - (1//2)*Cᵘ[3, 3]*(X^4)*ρₘ₀ + (1//4)*Cᵘ[3, 3]*(X^4)*ρₘ₁ + (1//4)*Cᵘ[3, 3]*(X^4)*ρₘ₋₁ + (1//4)*Cᵘ[3, 3]*(X^3)*Δx*ρₘ₁ - (1//4)*Cᵘ[3, 

In [81]:
# expression of (μu′)′

μu′ = simplify(newμ(X)*expand_derivatives(∂ₓ(u(X,T)));expand=true)
kinematics= simplify(expand_derivatives(∂ₓ(μu′));expand=true)

(-(2//1)*Cᵘ[2, 1]*X*μₘ₀ + Cᵘ[2, 1]*X*μₘ₁ + Cᵘ[2, 1]*X*μₘ₋₁ + (1//2)*Cᵘ[2, 1]*Δx*μₘ₁ - (1//2)*Cᵘ[2, 1]*Δx*μₘ₋₁ - (2//1)*Cᵘ[2, 2]*T*X*μₘ₀ + Cᵘ[2, 2]*T*X*μₘ₁ + Cᵘ[2, 2]*T*X*μₘ₋₁ + (1//2)*Cᵘ[2, 2]*T*Δx*μₘ₁ - (1//2)*Cᵘ[2, 2]*T*Δx*μₘ₋₁ - (3//1)*Cᵘ[3, 1]*(X^2)*μₘ₀ + (3//2)*Cᵘ[3, 1]*(X^2)*μₘ₁ + (3//2)*Cᵘ[3, 1]*(X^2)*μₘ₋₁ + Cᵘ[3, 1]*X*Δx*μₘ₁ - Cᵘ[3, 1]*X*Δx*μₘ₋₁ + Cᵘ[3, 1]*(Δx^2)*μₘ₀ - Cᵘ[2, 3]*(T^2)*X*μₘ₀ + (1//2)*Cᵘ[2, 3]*(T^2)*X*μₘ₁ + (1//2)*Cᵘ[2, 3]*(T^2)*X*μₘ₋₁ + (1//4)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₁ - (1//4)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₋₁ - (3//1)*Cᵘ[3, 2]*T*(X^2)*μₘ₀ + (3//2)*Cᵘ[3, 2]*T*(X^2)*μₘ₁ + (3//2)*Cᵘ[3, 2]*T*(X^2)*μₘ₋₁ + Cᵘ[3, 2]*T*X*Δx*μₘ₁ - Cᵘ[3, 2]*T*X*Δx*μₘ₋₁ + Cᵘ[3, 2]*T*(Δx^2)*μₘ₀ - (2//1)*Cᵘ[4, 1]*(X^3)*μₘ₀ + Cᵘ[4, 1]*(X^3)*μₘ₁ + Cᵘ[4, 1]*(X^3)*μₘ₋₁ + (3//4)*Cᵘ[4, 1]*(X^2)*Δx*μₘ₁ - (3//4)*Cᵘ[4, 1]*(X^2)*Δx*μₘ₋₁ + Cᵘ[4, 1]*X*(Δx^2)*μₘ₀ - (1//3)*Cᵘ[2, 4]*(T^3)*X*μₘ₀ + (1//6)*Cᵘ[2, 4]*(T^3)*X*μₘ₁ + (1//6)*Cᵘ[2, 4]*(T^3)*X*μₘ₋₁ + (1//12)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₁ - (1//12)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₋₁ -

In [82]:
# here we give the integrand
integrand=ρü-kinematics

((2//1)*Cᵘ[2, 1]*X*μₘ₀ - Cᵘ[2, 1]*X*μₘ₁ - Cᵘ[2, 1]*X*μₘ₋₁ - (1//2)*Cᵘ[2, 1]*Δx*μₘ₁ + (1//2)*Cᵘ[2, 1]*Δx*μₘ₋₁ + (2//1)*Cᵘ[2, 2]*T*X*μₘ₀ - Cᵘ[2, 2]*T*X*μₘ₁ - Cᵘ[2, 2]*T*X*μₘ₋₁ - (1//2)*Cᵘ[2, 2]*T*Δx*μₘ₁ + (1//2)*Cᵘ[2, 2]*T*Δx*μₘ₋₁ + (3//1)*Cᵘ[3, 1]*(X^2)*μₘ₀ - (3//2)*Cᵘ[3, 1]*(X^2)*μₘ₁ - (3//2)*Cᵘ[3, 1]*(X^2)*μₘ₋₁ - Cᵘ[3, 1]*X*Δx*μₘ₁ + Cᵘ[3, 1]*X*Δx*μₘ₋₁ - Cᵘ[3, 1]*(Δx^2)*μₘ₀ + Cᵘ[2, 3]*(T^2)*X*μₘ₀ - (1//2)*Cᵘ[2, 3]*(T^2)*X*μₘ₁ - (1//2)*Cᵘ[2, 3]*(T^2)*X*μₘ₋₁ - (1//4)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₁ + (1//4)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₋₁ + (3//1)*Cᵘ[3, 2]*T*(X^2)*μₘ₀ - (3//2)*Cᵘ[3, 2]*T*(X^2)*μₘ₁ - (3//2)*Cᵘ[3, 2]*T*(X^2)*μₘ₋₁ - Cᵘ[3, 2]*T*X*Δx*μₘ₁ + Cᵘ[3, 2]*T*X*Δx*μₘ₋₁ - Cᵘ[3, 2]*T*(Δx^2)*μₘ₀ + (2//1)*Cᵘ[4, 1]*(X^3)*μₘ₀ - Cᵘ[4, 1]*(X^3)*μₘ₁ - Cᵘ[4, 1]*(X^3)*μₘ₋₁ - (3//4)*Cᵘ[4, 1]*(X^2)*Δx*μₘ₁ + (3//4)*Cᵘ[4, 1]*(X^2)*Δx*μₘ₋₁ - Cᵘ[4, 1]*X*(Δx^2)*μₘ₀ + (1//3)*Cᵘ[2, 4]*(T^3)*X*μₘ₀ - (1//6)*Cᵘ[2, 4]*(T^3)*X*μₘ₁ - (1//6)*Cᵘ[2, 4]*(T^3)*X*μₘ₋₁ - (1//12)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₁ + (1//12)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₋₁ + 

In [83]:
f1_space=integrateTaylorPolynomials(integrand,X)
f2_space=integrateTaylorPolynomials(f1_space,X)

((1//3)*Cᵘ[2, 1]*(X^3)*μₘ₀ - (1//6)*Cᵘ[2, 1]*(X^3)*μₘ₁ - (1//6)*Cᵘ[2, 1]*(X^3)*μₘ₋₁ - (1//4)*Cᵘ[2, 1]*(X^2)*Δx*μₘ₁ + (1//4)*Cᵘ[2, 1]*(X^2)*Δx*μₘ₋₁ - (1//12)*Cᵘ[1, 3]*(X^4)*ρₘ₀ + (1//24)*Cᵘ[1, 3]*(X^4)*ρₘ₁ + (1//24)*Cᵘ[1, 3]*(X^4)*ρₘ₋₁ + (1//12)*Cᵘ[1, 3]*(X^3)*Δx*ρₘ₁ - (1//12)*Cᵘ[1, 3]*(X^3)*Δx*ρₘ₋₁ + (1//2)*Cᵘ[1, 3]*(X^2)*(Δx^2)*ρₘ₀ + (1//3)*Cᵘ[2, 2]*T*(X^3)*μₘ₀ - (1//6)*Cᵘ[2, 2]*T*(X^3)*μₘ₁ - (1//6)*Cᵘ[2, 2]*T*(X^3)*μₘ₋₁ - (1//4)*Cᵘ[2, 2]*T*(X^2)*Δx*μₘ₁ + (1//4)*Cᵘ[2, 2]*T*(X^2)*Δx*μₘ₋₁ + (1//4)*Cᵘ[3, 1]*(X^4)*μₘ₀ - (1//8)*Cᵘ[3, 1]*(X^4)*μₘ₁ - (1//8)*Cᵘ[3, 1]*(X^4)*μₘ₋₁ - (1//6)*Cᵘ[3, 1]*(X^3)*Δx*μₘ₁ + (1//6)*Cᵘ[3, 1]*(X^3)*Δx*μₘ₋₁ - (1//2)*Cᵘ[3, 1]*(X^2)*(Δx^2)*μₘ₀ - (1//12)*Cᵘ[1, 4]*T*(X^4)*ρₘ₀ + (1//24)*Cᵘ[1, 4]*T*(X^4)*ρₘ₁ + (1//24)*Cᵘ[1, 4]*T*(X^4)*ρₘ₋₁ + (1//12)*Cᵘ[1, 4]*T*(X^3)*Δx*ρₘ₁ - (1//12)*Cᵘ[1, 4]*T*(X^3)*Δx*ρₘ₋₁ + (1//2)*Cᵘ[1, 4]*T*(X^2)*(Δx^2)*ρₘ₀ + (1//6)*Cᵘ[2, 3]*(T^2)*(X^3)*μₘ₀ - (1//12)*Cᵘ[2, 3]*(T^2)*(X^3)*μₘ₁ - (1//12)*Cᵘ[2, 3]*(T^2)*(X^3)*μₘ₋₁ - (1//8)*Cᵘ[2, 3]*(

In [84]:
# this is F^{[2]} 
f2_space_right=simplify(substitute(f2_space,Dict([X=>Δx])))
f2_space_left=simplify(substitute(f2_space,Dict([X=>(-Δx)])))

-(1//3)*Cᵘ[2, 1]*Δx*μₘ₀ - (1//12)*Cᵘ[2, 1]*Δx*μₘ₁ + (5//12)*Cᵘ[2, 1]*Δx*μₘ₋₁ + (5//12)*Cᵘ[1, 3]*(Δx^2)*ρₘ₀ - (1//24)*Cᵘ[1, 3]*(Δx^2)*ρₘ₁ + (1//8)*Cᵘ[1, 3]*(Δx^2)*ρₘ₋₁ - (1//3)*Cᵘ[2, 2]*T*Δx*μₘ₀ - (1//12)*Cᵘ[2, 2]*T*Δx*μₘ₁ + (5//12)*Cᵘ[2, 2]*T*Δx*μₘ₋₁ - (1//4)*Cᵘ[3, 1]*(Δx^2)*μₘ₀ + (1//24)*Cᵘ[3, 1]*(Δx^2)*μₘ₁ - (7//24)*Cᵘ[3, 1]*(Δx^2)*μₘ₋₁ + (5//12)*Cᵘ[1, 4]*T*(Δx^2)*ρₘ₀ - (1//24)*Cᵘ[1, 4]*T*(Δx^2)*ρₘ₁ + (1//8)*Cᵘ[1, 4]*T*(Δx^2)*ρₘ₋₁ - (1//6)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₀ - (1//24)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₁ + (5//24)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₋₁ - (7//60)*Cᵘ[2, 3]*(Δx^3)*ρₘ₀ + (1//60)*Cᵘ[2, 3]*(Δx^3)*ρₘ₁ - (1//15)*Cᵘ[2, 3]*(Δx^3)*ρₘ₋₁ - (1//4)*Cᵘ[3, 2]*T*(Δx^2)*μₘ₀ + (1//24)*Cᵘ[3, 2]*T*(Δx^2)*μₘ₁ - (7//24)*Cᵘ[3, 2]*T*(Δx^2)*μₘ₋₁ + (1//15)*Cᵘ[4, 1]*(Δx^3)*μₘ₀ - (1//80)*Cᵘ[4, 1]*(Δx^3)*μₘ₁ + (9//80)*Cᵘ[4, 1]*(Δx^3)*μₘ₋₁ + (5//24)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₀ - (1//48)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₁ + (1//16)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₋₁ - (1//18)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₀ - (1//72)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₁ + (5//72)*Cᵘ[2, 

Here we want to evaluate:
\begin{equation}
-\left [ F^{[2]}(x) \phi^{(1)}(x) \right ]^{\Delta x}_{-\Delta  x}
\end{equation}
where we have 
\begin{align}
\phi^{(1)}(\Delta x) &=& -\frac{1}{\Delta x}\\
\phi^{(1)}(-\Delta x) &=& \frac{1}{\Delta x}
\end{align}


In [85]:
ϕ_deriv_right=-1//Δx
ϕ_deriv_left = 1//Δx
desired_value_in_space=-(ϕ_deriv_right*f2_space_right-ϕ_deriv_left*f2_space_left)

((1//3)*Cᵘ[2, 1]*Δx*μₘ₀ - (5//12)*Cᵘ[2, 1]*Δx*μₘ₁ + (1//12)*Cᵘ[2, 1]*Δx*μₘ₋₁ + (5//12)*Cᵘ[1, 3]*(Δx^2)*ρₘ₀ + (1//8)*Cᵘ[1, 3]*(Δx^2)*ρₘ₁ - (1//24)*Cᵘ[1, 3]*(Δx^2)*ρₘ₋₁ + (1//3)*Cᵘ[2, 2]*T*Δx*μₘ₀ - (5//12)*Cᵘ[2, 2]*T*Δx*μₘ₁ + (1//12)*Cᵘ[2, 2]*T*Δx*μₘ₋₁ - (1//4)*Cᵘ[3, 1]*(Δx^2)*μₘ₀ - (7//24)*Cᵘ[3, 1]*(Δx^2)*μₘ₁ + (1//24)*Cᵘ[3, 1]*(Δx^2)*μₘ₋₁ + (5//12)*Cᵘ[1, 4]*T*(Δx^2)*ρₘ₀ + (1//8)*Cᵘ[1, 4]*T*(Δx^2)*ρₘ₁ - (1//24)*Cᵘ[1, 4]*T*(Δx^2)*ρₘ₋₁ + (1//6)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₀ - (5//24)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₁ + (1//24)*Cᵘ[2, 3]*(T^2)*Δx*μₘ₋₁ + (7//60)*Cᵘ[2, 3]*(Δx^3)*ρₘ₀ + (1//15)*Cᵘ[2, 3]*(Δx^3)*ρₘ₁ - (1//60)*Cᵘ[2, 3]*(Δx^3)*ρₘ₋₁ - (1//4)*Cᵘ[3, 2]*T*(Δx^2)*μₘ₀ - (7//24)*Cᵘ[3, 2]*T*(Δx^2)*μₘ₁ + (1//24)*Cᵘ[3, 2]*T*(Δx^2)*μₘ₋₁ - (1//15)*Cᵘ[4, 1]*(Δx^3)*μₘ₀ - (9//80)*Cᵘ[4, 1]*(Δx^3)*μₘ₁ + (1//80)*Cᵘ[4, 1]*(Δx^3)*μₘ₋₁ + (5//24)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₀ + (1//16)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₁ - (1//48)*Cᵘ[1, 5]*(T^2)*(Δx^2)*ρₘ₋₁ + (1//18)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₀ - (5//72)*Cᵘ[2, 4]*(T^3)*Δx*μₘ₁ + (1//72)*Cᵘ[2, 

In [86]:
# now time integral

f1_time=integrateTaylorPolynomials(desired_value_in_space,T)
f2_time=integrateTaylorPolynomials(f1_time,T)


-(1//4)*Cᵘ[2, 1]*(T^2)*μₘ₁ + (1//4)*Cᵘ[2, 1]*(T^2)*μₘ₋₁ + (5//12)*Cᵘ[1, 3]*(T^2)*Δx*ρₘ₀ + (1//24)*Cᵘ[1, 3]*(T^2)*Δx*(ρₘ₁ + ρₘ₋₁) - (1//12)*Cᵘ[2, 2]*(T^3)*μₘ₁ + (1//12)*Cᵘ[2, 2]*(T^3)*μₘ₋₁ - (1//4)*Cᵘ[3, 1]*(T^2)*Δx*μₘ₀ - (1//8)*Cᵘ[3, 1]*(T^2)*Δx*(μₘ₁ + μₘ₋₁) + (5//36)*Cᵘ[1, 4]*(T^3)*Δx*ρₘ₀ + (1//72)*Cᵘ[1, 4]*(T^3)*Δx*(ρₘ₁ + ρₘ₋₁) - (1//48)*Cᵘ[2, 3]*(T^4)*μₘ₁ + (1//48)*Cᵘ[2, 3]*(T^4)*μₘ₋₁ + (1//24)*Cᵘ[2, 3]*(T^2)*(Δx^2)*ρₘ₁ - (1//24)*Cᵘ[2, 3]*(T^2)*(Δx^2)*ρₘ₋₁ - (1//12)*Cᵘ[3, 2]*(T^3)*Δx*μₘ₀ - (1//24)*Cᵘ[3, 2]*(T^3)*Δx*(μₘ₁ + μₘ₋₁) - (1//16)*Cᵘ[4, 1]*(T^2)*(Δx^2)*μₘ₁ + (1//16)*Cᵘ[4, 1]*(T^2)*(Δx^2)*μₘ₋₁ + (5//144)*Cᵘ[1, 5]*(T^4)*Δx*ρₘ₀ + (1//288)*Cᵘ[1, 5]*(T^4)*Δx*(ρₘ₁ + ρₘ₋₁) - (1//240)*Cᵘ[2, 4]*(T^5)*μₘ₁ + (1//240)*Cᵘ[2, 4]*(T^5)*μₘ₋₁ + (1//72)*Cᵘ[2, 4]*(T^3)*(Δx^2)*ρₘ₁ - (1//72)*Cᵘ[2, 4]*(T^3)*(Δx^2)*ρₘ₋₁ - (1//48)*Cᵘ[3, 3]*(T^4)*Δx*μₘ₀ - (1//96)*Cᵘ[3, 3]*(T^4)*Δx*(μₘ₁ + μₘ₋₁) + (1//40)*Cᵘ[3, 3]*(T^2)*(Δx^3)*ρₘ₀ + (1//120)*Cᵘ[3, 3]*(T^2)*(Δx^3)*(ρₘ₁ + ρₘ₋₁) - (1//48)*Cᵘ[4, 2]*(T^3)*(

In [87]:
f2_time_right=simplify(substitute(f2_time,Dict([T=>Δt])))
f2_time_left=simplify(substitute(f2_time,Dict([T=>(-Δt)])))

-(1//4)*Cᵘ[2, 1]*(Δt^2)*μₘ₁ + (1//4)*Cᵘ[2, 1]*(Δt^2)*μₘ₋₁ + (5//12)*Cᵘ[1, 3]*(Δt^2)*Δx*ρₘ₀ + (1//24)*Cᵘ[1, 3]*(Δt^2)*Δx*(ρₘ₁ + ρₘ₋₁) + (1//12)*Cᵘ[2, 2]*(Δt^3)*μₘ₁ - (1//12)*Cᵘ[2, 2]*(Δt^3)*μₘ₋₁ - (1//4)*Cᵘ[3, 1]*(Δt^2)*Δx*μₘ₀ - (1//8)*Cᵘ[3, 1]*(Δt^2)*Δx*(μₘ₁ + μₘ₋₁) - (5//36)*Cᵘ[1, 4]*(Δt^3)*Δx*ρₘ₀ - (1//72)*Cᵘ[1, 4]*(Δt^3)*Δx*(ρₘ₁ + ρₘ₋₁) - (1//48)*Cᵘ[2, 3]*(Δt^4)*μₘ₁ + (1//48)*Cᵘ[2, 3]*(Δt^4)*μₘ₋₁ + (1//24)*Cᵘ[2, 3]*(Δt^2)*(Δx^2)*ρₘ₁ - (1//24)*Cᵘ[2, 3]*(Δt^2)*(Δx^2)*ρₘ₋₁ + (1//12)*Cᵘ[3, 2]*(Δt^3)*Δx*μₘ₀ + (1//24)*Cᵘ[3, 2]*(Δt^3)*Δx*(μₘ₁ + μₘ₋₁) - (1//16)*Cᵘ[4, 1]*(Δt^2)*(Δx^2)*μₘ₁ + (1//16)*Cᵘ[4, 1]*(Δt^2)*(Δx^2)*μₘ₋₁ + (5//144)*Cᵘ[1, 5]*(Δt^4)*Δx*ρₘ₀ + (1//288)*Cᵘ[1, 5]*(Δt^4)*Δx*(ρₘ₁ + ρₘ₋₁) + (1//240)*Cᵘ[2, 4]*(Δt^5)*μₘ₁ - (1//240)*Cᵘ[2, 4]*(Δt^5)*μₘ₋₁ - (1//72)*Cᵘ[2, 4]*(Δt^3)*(Δx^2)*ρₘ₁ + (1//72)*Cᵘ[2, 4]*(Δt^3)*(Δx^2)*ρₘ₋₁ - (1//48)*Cᵘ[3, 3]*(Δt^4)*Δx*μₘ₀ - (1//96)*Cᵘ[3, 3]*(Δt^4)*Δx*(μₘ₁ + μₘ₋₁) + (1//40)*Cᵘ[3, 3]*(Δt^2)*(Δx^3)*ρₘ₀ + (1//120)*Cᵘ[3, 3]*(Δt^2)*(Δx^3)*(ρₘ₁ + ρₘ₋₁

In [88]:
ψ_deriv_right=-1//Δt
ψ_deriv_left = 1//Δt
desired_value_in_time=-(ψ_deriv_right*f2_time_right-ψ_deriv_left*f2_time_left)
desired_value_in_time=simplify(desired_value_in_time;expand=true)
#desired_value_in_time=ignoreHigherOrderTerms(desired_value_in_time,Δx,2)
#desired_value_in_time=ignoreHigherOrderTerms(desired_value_in_time,Δt,2)

-(1//2)*Cᵘ[2, 1]*Δt*μₘ₁ + (1//2)*Cᵘ[2, 1]*Δt*μₘ₋₁ + (5//6)*Cᵘ[1, 3]*Δt*Δx*ρₘ₀ + (1//12)*Cᵘ[1, 3]*Δt*Δx*ρₘ₁ + (1//12)*Cᵘ[1, 3]*Δt*Δx*ρₘ₋₁ - (1//2)*Cᵘ[3, 1]*Δt*Δx*μₘ₀ - (1//4)*Cᵘ[3, 1]*Δt*Δx*μₘ₁ - (1//4)*Cᵘ[3, 1]*Δt*Δx*μₘ₋₁ - (1//24)*Cᵘ[2, 3]*(Δt^3)*μₘ₁ + (1//24)*Cᵘ[2, 3]*(Δt^3)*μₘ₋₁ + (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₁ - (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₋₁ - (1//8)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ + (1//8)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁ + (5//72)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₀ + (1//144)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₁ + (1//144)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₋₁ - (1//24)*Cᵘ[3, 3]*(Δt^3)*Δx*μₘ₀ - (1//48)*Cᵘ[3, 3]*(Δt^3)*Δx*μₘ₁ - (1//48)*Cᵘ[3, 3]*(Δt^3)*Δx*μₘ₋₁ + (1//20)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₀ + (1//60)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₁ + (1//60)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₋₁ - (1//36)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₀ - (1//36)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₁ - (1//36)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₋₁ - (1//720)*Cᵘ[2, 5]*(Δt^5)*μₘ₁ + (1//720)*Cᵘ[2, 5]*(Δt^5)*μₘ₋₁ + (1//144)*Cᵘ[2, 5]*(Δt^3)*(Δx^2)*ρₘ₁ - (1//144)*Cᵘ[2, 5]*(Δt^3)*(Δx^2)*ρₘ₋₁ - (1//96)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₁ + (1//

\begin{equation}
 \sum_{m,n} A_{mn} u_{mn}
 \end{equation}

In [89]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=Aₘₙ[i¹,iᵗ]*u(dx¹,dt)
    end
end
difference_between_matrix_strongform=simplify(innerProduct-desired_value_in_time;expand=true)
Symbolics.coeff(difference_between_matrix_strongform,Cᵘ[2,3])

(1//2)*Aₘₙ₁ˏ₁*(Δt^2)*Δx + (1//2)*Aₘₙ₁ˏ₋₁*(Δt^2)*Δx - (1//2)*Aₘₙ₋₁ˏ₁*(Δt^2)*Δx - (1//2)*Aₘₙ₋₁ˏ₋₁*(Δt^2)*Δx + (1//24)*(Δt^3)*μₘ₁ - (1//24)*(Δt^3)*μₘ₋₁ - (1//12)*Δt*(Δx^2)*ρₘ₁ + (1//12)*Δt*(Δx^2)*ρₘ₋₁

In [90]:
hierarchical_equations=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1   
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1  
        push!(hierarchical_equations,Symbolics.coeff(difference_between_matrix_strongform,Cᵘ[i¹,iᵗ])~0)
    end
end

In [91]:

for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(hierarchical_equations[(i¹-1)*(Lₜₗ+Lₜᵣ+1)+iᵗ]))
    end
end

Aₘₙ₀ˏ₀ + Aₘₙ₀ˏ₁ + Aₘₙ₀ˏ₋₁ + Aₘₙ₁ˏ₀ + Aₘₙ₁ˏ₁ + Aₘₙ₁ˏ₋₁ + Aₘₙ₋₁ˏ₀ + Aₘₙ₋₁ˏ₁ + Aₘₙ₋₁ˏ₋₁ ~ 0

(Aₘₙ₀ˏ₁ + Aₘₙ₁ˏ₁ + Aₘₙ₋₁ˏ₁)*Δt - Aₘₙ₀ˏ₋₁*Δt - Aₘₙ₁ˏ₋₁*Δt - Aₘₙ₋₁ˏ₋₁*Δt ~ 0

(1//2)*Aₘₙ₀ˏ₁*(Δt^2) + (1//2)*Aₘₙ₀ˏ₋₁*(Δt^2) + (1//2)*Aₘₙ₁ˏ₁*(Δt^2) + (1//2)*Aₘₙ₁ˏ₋₁*(Δt^2) + (1//2)*Aₘₙ₋₁ˏ₁*(Δt^2) + (1//2)*Aₘₙ₋₁ˏ₋₁*(Δt^2) - (5//6)*Δt*Δx*ρₘ₀ - (1//12)*Δt*Δx*(ρₘ₁ + ρₘ₋₁) ~ 0

(Aₘₙ₁ˏ₀ + Aₘₙ₁ˏ₁ + Aₘₙ₁ˏ₋₁)*Δx - Aₘₙ₋₁ˏ₀*Δx - Aₘₙ₋₁ˏ₁*Δx - Aₘₙ₋₁ˏ₋₁*Δx + (1//2)*Δt*μₘ₁ - (1//2)*Δt*μₘ₋₁ ~ 0

(Aₘₙ₁ˏ₁ + Aₘₙ₋₁ˏ₋₁)*Δt*Δx - Aₘₙ₁ˏ₋₁*Δt*Δx - Aₘₙ₋₁ˏ₁*Δt*Δx ~ 0

(1//2)*Aₘₙ₁ˏ₁*(Δt^2)*Δx + (1//2)*Aₘₙ₁ˏ₋₁*(Δt^2)*Δx - (1//2)*Aₘₙ₋₁ˏ₁*(Δt^2)*Δx - (1//2)*Aₘₙ₋₁ˏ₋₁*(Δt^2)*Δx + (1//24)*(Δt^3)*μₘ₁ - (1//24)*(Δt^3)*μₘ₋₁ - (1//12)*Δt*(Δx^2)*ρₘ₁ + (1//12)*Δt*(Δx^2)*ρₘ₋₁ ~ 0

(1//2)*Aₘₙ₁ˏ₀*(Δx^2) + (1//2)*Aₘₙ₁ˏ₁*(Δx^2) + (1//2)*Aₘₙ₁ˏ₋₁*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₀*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₁*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₋₁*(Δx^2) + (1//2)*Δt*Δx*μₘ₀ + (1//4)*Δt*Δx*(μₘ₁ + μₘ₋₁) ~ 0

(1//2)*Aₘₙ₁ˏ₁*Δt*(Δx^2) - (1//2)*Aₘₙ₁ˏ₋₁*Δt*(Δx^2) + (1//2)*Aₘₙ₋₁ˏ₁*Δt*(Δx^2) - (1//2)*Aₘₙ₋₁ˏ₋₁*Δt*(Δx^2) ~ 0

(1//4)*Aₘₙ₁ˏ₁*(Δt^2)*(Δx^2) + (1//4)*Aₘₙ₁ˏ₋₁*(Δt^2)*(Δx^2) + (1//4)*Aₘₙ₋₁ˏ₁*(Δt^2)*(Δx^2) + (1//4)*Aₘₙ₋₁ˏ₋₁*(Δt^2)*(Δx^2) + (1//24)*(Δt^3)*Δx*μₘ₀ + (1//48)*(Δt^3)*Δx*(μₘ₁ + μₘ₋₁) - (1//20)*Δt*(Δx^3)*ρₘ₀ - (1//60)*Δt*(Δx^3)*(ρₘ₁ + ρₘ₋₁) ~ 0

In [92]:

newA=reshape(Symbolics.solve_for(hierarchical_equations,Aₘₙ),(L₁ₗ+L₁ᵣ+1,Lₜₗ+Lₜᵣ+1))
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(newA[i¹,iᵗ];expand=true))
    end
end

(-(1//24)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₋₁ + (1//20)*(Δx^2)*ρₘ₀ - (1//40)*(Δx^2)*ρₘ₁ + (7//120)*(Δx^2)*ρₘ₋₁) / (Δt*Δx)

0

0

(-(1//12)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₁ - (1//24)*(Δt^2)*μₘ₋₁ - (11//15)*(Δx^2)*ρₘ₀ - (1//20)*(Δx^2)*ρₘ₁ - (1//20)*(Δx^2)*ρₘ₋₁) / (-Δt*Δx)

(-(5//6)*(Δt^2)*μₘ₀ - (5//12)*(Δt^2)*μₘ₁ - (5//12)*(Δt^2)*μₘ₋₁ + (22//15)*(Δx^2)*ρₘ₀ + (1//10)*(Δx^2)*ρₘ₁ + (1//10)*(Δx^2)*ρₘ₋₁) / (-Δt*Δx)

(-(1//12)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₁ - (1//24)*(Δt^2)*μₘ₋₁ - (11//15)*(Δx^2)*ρₘ₀ - (1//20)*(Δx^2)*ρₘ₁ - (1//20)*(Δx^2)*ρₘ₋₁) / (-Δt*Δx)

0

((5//12)*(Δt^2)*μₘ₀ + (5//12)*(Δt^2)*μₘ₁ + (1//10)*(Δx^2)*ρₘ₀ + (7//60)*(Δx^2)*ρₘ₁ - (1//20)*(Δx^2)*ρₘ₋₁) / (-Δt*Δx)

((1//24)*(Δt^2)*μₘ₀ + (1//24)*(Δt^2)*μₘ₁ - (1//20)*(Δx^2)*ρₘ₀ - (7//120)*(Δx^2)*ρₘ₁ + (1//40)*(Δx^2)*ρₘ₋₁) / (-Δt*Δx)

Now I need to verify by evaluating the error

In [94]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=newA[i¹,iᵗ]*u(dx¹,dt)
    end
end
difference=simplify(innerProduct-desired_value_in_time,expand=true)
display(difference)
truncated_difference=ignoreHigherOrderTerms(difference,Δx,2)
truncated_difference=ignoreHigherOrderTerms(truncated_difference,Δt,2)
display(truncated_difference)

(1//20736)*((864//1)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ - (864//1)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁ - (288//1)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₀ + (144//1)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₁ + (144//1)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₋₁ - (43//1)*Cᵘ[2, 5]*(Δt^5)*μₘ₁ + (43//1)*Cᵘ[2, 5]*(Δt^5)*μₘ₋₁ + (72//1)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₁ - (72//1)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₋₁ + (172//1)*Cᵘ[4, 3]*Δt*(Δx^4)*ρₘ₁ - (172//1)*Cᵘ[4, 3]*Δt*(Δx^4)*ρₘ₋₁ - (43//1)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₀ - (21//1)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₁ - (21//1)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₋₁ - (24//1)*Cᵘ[5, 3]*(Δt^3)*(Δx^3)*μₘ₀ + (12//1)*Cᵘ[5, 3]*(Δt^3)*(Δx^3)*μₘ₁ + (12//1)*Cᵘ[5, 3]*(Δt^3)*(Δx^3)*μₘ₋₁ + (59//1)*Cᵘ[5, 3]*Δt*(Δx^5)*ρₘ₀ + (13//1)*Cᵘ[5, 3]*Δt*(Δx^5)*ρₘ₁ + (13//1)*Cᵘ[5, 3]*Δt*(Δx^5)*ρₘ₋₁ - (4//1)*Cᵘ[4, 5]*(Δt^5)*(Δx^2)*μₘ₁ + (4//1)*Cᵘ[4, 5]*(Δt^5)*(Δx^2)*μₘ₋₁ + (14//1)*Cᵘ[4, 5]*(Δt^3)*(Δx^4)*ρₘ₁ - (14//1)*Cᵘ[4, 5]*(Δt^3)*(Δx^4)*ρₘ₋₁ - (4//1)*Cᵘ[5, 5]*(Δt^5)*(Δx^3)*μₘ₀ - Cᵘ[5, 5]*(Δt^5)*(Δx^3)*μₘ₁ - Cᵘ[5, 5]*(Δt^5)*(Δx^3)*μₘ₋₁ + (4//1)*Cᵘ[5, 5]*(Δt^3)*(Δx^5)*ρₘ₀ + Cᵘ[5, 5]*(Δt^3)*(Δx^

(1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ - (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁

# Test classical FD operator

In [95]:
A_classic=Symbolics.variables(:A_classic,-L₁ₗ:L₁ᵣ,-Lₜₗ:Lₜᵣ)
A_classic.=0
A_classic[2,1]=ρₘ[2]//Δt^2
A_classic[2,2]=-2*ρₘ[2]//Δt^2 + (μₘ[1]+2*μₘ[2]+μₘ[3])//(2*Δx^2)
A_classic[2,3]=ρₘ[2]//Δt^2

A_classic[1,2] = -(μₘ[1]+μₘ[2])//(2*Δx^2)
A_classic[3,2] = -(μₘ[2]+μₘ[3])//(2*Δx^2)

A_classic=A_classic*Δx*Δt

for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(A_classic[i¹,iᵗ];expand=true))
    end
end

0

(Δt*(-μₘ₀ - μₘ₋₁)) / (2Δx)

0

(Δx*ρₘ₀) / Δt

(2(Δt^2)*μₘ₀ + (Δt^2)*μₘ₁ + (Δt^2)*μₘ₋₁ - 4(Δx^2)*ρₘ₀) / (2Δt*Δx)

(Δx*ρₘ₀) / Δt

0

(Δt*(-μₘ₀ - μₘ₁)) / (2Δx)

0

In [96]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=A_classic[i¹,iᵗ]*u(dx¹,dt)
    end
end
differenceC=simplify(innerProduct-desired_value_in_time,expand=true)
display(differenceC)
truncated_differenceC=ignoreHigherOrderTerms(differenceC,Δx,2)
truncated_differenceC=ignoreHigherOrderTerms(truncated_differenceC,Δt,2)
display(truncated_differenceC)

(1//8)*((4//3)*Cᵘ[1, 3]*Δt*Δx*ρₘ₀ - (2//3)*Cᵘ[1, 3]*Δt*Δx*ρₘ₁ - (2//3)*Cᵘ[1, 3]*Δt*Δx*ρₘ₋₁ + (1//3)*Cᵘ[2, 3]*(Δt^3)*μₘ₁ - (1//3)*Cᵘ[2, 3]*(Δt^3)*μₘ₋₁ - (2//3)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₁ + (2//3)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₋₁ + (1//3)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ - (1//3)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁ + (1//9)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₀ - (1//18)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₁ - (1//18)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₋₁ + (1//3)*Cᵘ[3, 3]*(Δt^3)*Δx*μₘ₀ + (1//6)*Cᵘ[3, 3]*(Δt^3)*Δx*μₘ₁ + (1//6)*Cᵘ[3, 3]*(Δt^3)*Δx*μₘ₋₁ - (2//5)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₀ - (2//15)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₁ - (2//15)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₋₁ - (1//9)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₀ + (1//18)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₁ + (1//18)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₋₁ + (1//90)*Cᵘ[2, 5]*(Δt^5)*μₘ₁ - (1//90)*Cᵘ[2, 5]*(Δt^5)*μₘ₋₁ - (1//18)*Cᵘ[2, 5]*(Δt^3)*(Δx^2)*ρₘ₁ + (1//18)*Cᵘ[2, 5]*(Δt^3)*(Δx^2)*ρₘ₋₁ + (1//12)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₁ - (1//12)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₋₁ - (2//45)*Cᵘ[4, 3]*Δt*(Δx^4)*ρₘ₁ + (2//45)*Cᵘ[4, 3]*Δt*(Δx^4)*ρₘ₋₁ + (1//90)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₀ + (1//180)*Cᵘ[3

(1//6)*Cᵘ[1, 3]*Δt*Δx*ρₘ₀ - (1//12)*Cᵘ[1, 3]*Δt*Δx*(ρₘ₁ + ρₘ₋₁) - (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₁ + (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₋₁ + (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ - (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁

# Test GT98 operator

In [97]:
A_GT98[1,1]=ρₘ[2]//12//Δt^2-(μₘ[1]+μₘ[2])//24//Δx^2
A_GT98[1,2]=-2ρₘ[2]//12//Δt^2-10*(μₘ[1]+μₘ[2])//24//Δx^2
A_GT98[1,3]=ρₘ[2]//12//Δt^2-(μₘ[1]+μₘ[2])//24//Δx^2

A_GT98[2,1]=10*ρₘ[2]//12//Δt^2+(μₘ[1]+2*μₘ[2]+μₘ[3])//24//Δx^2
A_GT98[2,2]=-20*ρₘ[2]//12//Δt^2+10*(μₘ[1]+2*μₘ[2]+μₘ[3])//24//Δx^2
A_GT98[2,3]=10*ρₘ[2]//12//Δt^2+(μₘ[1]+2*μₘ[2]+μₘ[3])//24//Δx^2

A_GT98[3,1]=ρₘ[2]//12//Δt^2-(μₘ[3]+μₘ[2])//24//Δx^2
A_GT98[3,2]=-2ρₘ[2]//12//Δt^2-10*(μₘ[3]+μₘ[2])//24//Δx^2
A_GT98[3,3]=ρₘ[2]//12//Δt^2-(μₘ[3]+μₘ[2])//24//Δx^2
A_GT98=A_GT98*Δx*Δt
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        display(simplify(A_GT98[i¹,iᵗ];expand=true))
    end
end

(-(1//24)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₋₁ + (1//12)*(Δx^2)*ρₘ₀) / (Δt*Δx)

(-(5//12)*(Δt^2)*μₘ₀ - (5//12)*(Δt^2)*μₘ₋₁ - (1//6)*(Δx^2)*ρₘ₀) / (Δt*Δx)

(-(1//24)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₋₁ + (1//12)*(Δx^2)*ρₘ₀) / (Δt*Δx)

((1//12)*(Δt^2)*μₘ₀ + (1//24)*(Δt^2)*μₘ₁ + (1//24)*(Δt^2)*μₘ₋₁ + (5//6)*(Δx^2)*ρₘ₀) / (Δt*Δx)

((5//6)*(Δt^2)*μₘ₀ + (5//12)*(Δt^2)*μₘ₁ + (5//12)*(Δt^2)*μₘ₋₁ - (5//3)*(Δx^2)*ρₘ₀) / (Δt*Δx)

((1//12)*(Δt^2)*μₘ₀ + (1//24)*(Δt^2)*μₘ₁ + (1//24)*(Δt^2)*μₘ₋₁ + (5//6)*(Δx^2)*ρₘ₀) / (Δt*Δx)

(-(1//24)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₁ + (1//12)*(Δx^2)*ρₘ₀) / (Δt*Δx)

(-(5//12)*(Δt^2)*μₘ₀ - (5//12)*(Δt^2)*μₘ₁ - (1//6)*(Δx^2)*ρₘ₀) / (Δt*Δx)

(-(1//24)*(Δt^2)*μₘ₀ - (1//24)*(Δt^2)*μₘ₁ + (1//12)*(Δx^2)*ρₘ₀) / (Δt*Δx)

In [98]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        innerProduct+=A_GT98[i¹,iᵗ]*u(dx¹,dt)
    end
end
differenceGT=simplify(innerProduct-desired_value_in_time,expand=true)
display(differenceGT)
truncated_differenceGT=ignoreHigherOrderTerms(differenceGT,Δx,2)
truncated_differenceGT=ignoreHigherOrderTerms(truncated_differenceGT,Δt,2)
display(truncated_differenceGT)

(1//6)*Cᵘ[1, 3]*Δt*Δx*ρₘ₀ - (1//12)*Cᵘ[1, 3]*Δt*Δx*ρₘ₁ - (1//12)*Cᵘ[1, 3]*Δt*Δx*ρₘ₋₁ - (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₁ + (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₋₁ + (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ - (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁ + (1//72)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₀ - (1//144)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₁ - (1//144)*Cᵘ[1, 5]*(Δt^3)*Δx*ρₘ₋₁ + (1//30)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₀ - (1//60)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₁ - (1//60)*Cᵘ[3, 3]*Δt*(Δx^3)*ρₘ₋₁ - (1//72)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₀ + (1//144)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₁ + (1//144)*Cᵘ[5, 1]*Δt*(Δx^3)*μₘ₋₁ - (1//480)*Cᵘ[2, 5]*(Δt^5)*μₘ₁ + (1//480)*Cᵘ[2, 5]*(Δt^5)*μₘ₋₁ - (1//144)*Cᵘ[2, 5]*(Δt^3)*(Δx^2)*ρₘ₁ + (1//144)*Cᵘ[2, 5]*(Δt^3)*(Δx^2)*ρₘ₋₁ + (1//288)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₁ - (1//288)*Cᵘ[4, 3]*(Δt^3)*(Δx^2)*μₘ₋₁ - (1//180)*Cᵘ[4, 3]*Δt*(Δx^4)*ρₘ₁ + (1//180)*Cᵘ[4, 3]*Δt*(Δx^4)*ρₘ₋₁ - (1//480)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₀ - (1//960)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₁ - (1//960)*Cᵘ[3, 5]*(Δt^5)*Δx*μₘ₋₁ + (1//360)*Cᵘ[3, 5]*(Δt^3)*(Δx^3)*ρₘ₀ - (1//720)*Cᵘ[3, 5]*(Δt^3)*(Δx^3)*ρₘ₁ - (1//720)*C

(1//6)*Cᵘ[1, 3]*Δt*Δx*ρₘ₀ - (1//12)*Cᵘ[1, 3]*Δt*Δx*(ρₘ₁ + ρₘ₋₁) - (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₁ + (1//12)*Cᵘ[2, 3]*Δt*(Δx^2)*ρₘ₋₁ + (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₁ - (1//24)*Cᵘ[4, 1]*Δt*(Δx^2)*μₘ₋₁

In [ ]:
# trial functions
function TruncatedSinc(x,scaling_factor=1)
    # if scaling_factor is 1, then the sinc function is scaled by π and has 0 every node=1
    #  (i.e. the sinc function is sin(π*x)/(π*x)
    # if we change the scaling_factor then the node interval is multiplied by the scaling_factor
    # if scaling_factor is 0, then the sinc function is unscaled (i.e. the sinc function is sin(x)/x)
    if scaling_factor == 0
        scaling_factor = π
    end 
    x=x/scaling_factor
    IfElse.ifelse(x==0,1,sin(π*x)/(π*x))
end

function Spline(x,j,k,nodes)
    IfElse.ifelse(k>0,
        IfElse.ifelse(j+1 <= length(nodes),
            (x-nodes[j])/(nodes[j+k]-nodes[j])*Spline(x,j,k-1,nodes),
            0
        )+
        IfElse.ifelse(j+k+1 <= length(nodes),
            (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*Spline(x,j+1,k-1,nodes),
            0
        ),
        IfElse.ifelse(j>=1 && j+1 <=length(nodes) && nodes[j] <= x < nodes[j+1],
            1,
            0
        )
    )
    """
    if k>0 
        if j+1 <= length(nodes)
            X_local += (x-nodes[j])/(nodes[j+k]-nodes[j])*Spline(x,j,k-1,nodes) 
        end
        if j+k+1 <= length(nodes)
            X_local += X_local + (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*Spline(x,j+1,k-1,nodes)
        end      
    elseif k==0
        if j>=1 && j+1 <=length(nodes) && nodes[j] <= x < nodes[j+1]
            X_local = 1.e0
        end
    end
    """
end

In [ ]:
ϕᵐ(x¹) = Spline(x¹,L₁ₗ+1,1,ξ¹) # x₁ direction linear spline function
ψⁿ(t) = Spline(t,Lₜₗ+1,1,τ)

In [ ]:
ϕᵐ(x¹)

In [ ]:
u(x¹-ξ¹[L₁ₗ+1] , t-τ[Lₜₗ+1])

In [ ]:
ρ(x¹-ξ¹[L₁ₗ+1])

In [ ]:
@variables x t Δx Δt f ρₒ μₒ uₒ
@syms ρ(x) μ(x)
∂ₜ = Differential(t)
∂ₓ = Differential(x)

Taylorₓ(f,order) = f+sum((∂ₓ^i)(f(x))*Δx^i/factorial(BigInt(i)) for i in 1:order)
Taylorₜ(f,order) = f+sum((∂ₜ^i)f*Δt^i/factorial(BigInt(i)) for i in 1:order)
Taylorₓₜ(f,order) = f+sum((∂ₓ^i)(f)*Δx^i/factorial(BigInt(i)) for i in 1:order) + sum((∂ₜ^i)(f)*Δt^i/factorial(BigInt(i)) for i in 1:order) +sum(sum((∂ₜ^k)((∂ₓ^i)(f))*Δx^i/factorial(BigInt(i))*Δt^k/factorial(BigInt(k)) for i in 1:order) for k in 1:order)
DerivativeₓTaylorₓ(f,order) = sum((∂ₓ^i)(f)*Δx^(i-1)/factorial(BigInt(i-1)) for i in 1:order)
DerivativeₜTaylorₜ(f,order) = sum((∂ₓ^i)(f)*Δt^(i-1)/factorial(BigInt(i-1)) for i in 1:order)
DerivativeₓTaylorₓₜ(f,order) = sum((∂ₓ^i)(f)*Δx^(i-1)/factorial(BigInt(i-1)) for i in 1:order) + +sum(sum((∂ₜ^k)((∂ₓ^i)(f))*Δx^(i-1)/factorial(BigInt(i-1))*Δt^k/factorial(BigInt(k)) for i in 1:order) for k in 1:order)
DerivativeₜTaylorₓₜ(f,order) = sum((∂ₜ^i)(f)*Δt^(i-1)/factorial(BigInt(i-1)) for i in 1:order) + +sum(sum((∂ₜ^k)((∂ₓ^i)(f))*Δx^i/factorial(BigInt(i-1))*Δt^(k-1)/factorial(BigInt(k-1)) for i in 1:order) for k in 1:order)

In [ ]:
@variables xₘ tₙ
u=substitute(Taylorₓₜ(uₒ,4),Dict(Δx=>x-xₘ, Δt=>t-tₙ))
ρ=substitute(Taylorₓ(ρ₀,4),Dict(Δx=>x-xₘ))
μ=substitute(Taylorₓ(μ₀,4),Dict(Δx=>x-xₘ))
u₁=substitute(DerivativeₓTaylorₓₜ(uₒ,4),Dict(Δx=>x-xₘ, Δt=>t-tₙ))


In [ ]:
@syms ρ(x,t) xₘ
tmpTaylor(f,order)=(∂ₓ^2)(f(x,t)) |>expand_derivatives
z(x,t)=tmpTaylor(ρ,2)
z(xₘ,t)


In [ ]:
Differential(x)(x^2) |>expand_derivatives